# Homework 5
*Edited by Yibang (Christopher) Liu, 8/22*

# Question 1 Algorithmic questions

Find the kth largest element in an unsorted array. Note that it is the kth largest element in the sorted order, not the kth distinct element.

## Examples:

* Input: [3,2,1,5,6,4] and k = 2

* Output: 5

* Input: [3,2,3,1,2,4,5,5,6] and k = 4
* Output: 4

You may assume k is always valid, 1 ≤ k ≤ array's length.

To receive full credit, you should do the following:

* Develop several test cases and explain why you choose these test cases
* Wrap your solution into a function.
* Analyze the time complexity of your algorithm.

Hint: You should use heapq to solve the question, but you should not use heapq.nlargest.

In [1]:
import heapq

def k_largest(nums, k):
    if k > len(nums) or k < 1:
        raise IndexError("k out of range or empty array")
    
    # Heapify the first k elements of the array
    heap = nums[:k]
    heapq.heapify(heap)
    
    # For every element, substitute it for the heap root
    # if it's larger and maintain the heap
    for num in nums[k:]:
        heapq.heappushpop(heap, num)
    
    # Heap root is the kth largest element
    return heap[0]

* Time complexity of the solution:
    * Heapify the first k elements: O(k)
    * Compare and replace for (n-k) elements: O((n-k)logk)
    * Total time complexity: O(k) + O((n-k)logk)
    * When k << n: O(nlogk)
    * When k very close to n: O(n)

In [2]:
# Test case 1
# Invalid k
nums = [1]
k = 2
k_largest(nums, k)

IndexError: k out of range or empty array

In [3]:
# Test case 2
# All elements distinct
nums = [3,2,1,5,6,4]
k = 2
print(k_largest(nums, k))
print(heapq.nlargest(k, nums)[-1])

5
5


In [4]:
# Test case 3
# Multiple kth largest elements exist
nums = [3,2,3,1,2,3,5,5,6]
k = 4
print(k_largest(nums, k))
print(heapq.nlargest(k, nums)[-1])

3
3


In [5]:
# Test case 4
# Edge case: 1 element
nums = [0]
k = 1
print(k_largest(nums, k))
print(heapq.nlargest(k, nums)[-1])

0
0


In [6]:
# Test case 5
# Large case
import random

nums = [random.randint(1, 100000) for i in range(10000)]
k = 200
print(k_largest(nums, k))
print(heapq.nlargest(k, nums)[-1])

97857
97857


# Question 2 Multithreading pop.

Using python data structure Queue, generate a queue that have 10000 elments. Then use 1, 5, 20 threads to pop out the elments out. Compare the efficiency when you use different number of threads and explain why you see the pattern of performance.

In [7]:
from queue import Queue

def init_queue(n):
    q = Queue()
    for i in range(n):
        q.put(i)
    return q

In [8]:
from threading import Thread, Lock
import time

def multithread_pure_pop(q, thread_num):
    thread_group = []
    
    def pop_item(q_, n):
        while n:
            n -= 1
            q_.get()
    
    for i in range(thread_num):
        t = Thread(target=pop_item, args=(q, 10000 // thread_num))
        thread_group.append(t)
        t.start()
    
    for t in thread_group:
        t.join()

q = init_queue(10000)
%time multithread_pure_pop(q, 1)
q = init_queue(10000)
%time multithread_pure_pop(q, 5)
q = init_queue(10000)
%time multithread_pure_pop(q, 20)

Wall time: 15 ms
Wall time: 15 ms
Wall time: 16 ms


In [9]:
def multithread_computation(q, thread_num):
    thread_group = []
    lock = Lock()
    
    def pop_item(q_):
        while True:
            # Lock is needed to ensure that no elements are popped
            # by other threads between empty() and get()
            lock.acquire()
            if not q_.empty():
                q_.get()
                lock.release()
            else:
                lock.release()
                break
            # Simulating computation based on the popped element
            time.sleep(0.001) # The smallest sleeping time we can do
    
    for i in range(thread_num):
        t = Thread(target=pop_item, args=(q,))
        thread_group.append(t)
        t.start()
    
    for t in thread_group:
        t.join()

q = init_queue(10000)
%time multithread_computation(q, 1)
q = init_queue(10000)
%time multithread_computation(q, 5)
q = init_queue(10000)
%time multithread_computation(q, 20)

Wall time: 19.9 s
Wall time: 3.99 s
Wall time: 999 ms


* For pure pop without computation, time consumption doesn't vary a lot when the number of threads increases, this is because:
    * Queue is thread safe and Python uses GIL, meaning that the get(block=True) action cannot be performed simultaniously by multiple threads;
    * Overhead is not to be neglected since get() is a pretty light task.

* For pop with computation that doesn't involve memory share, multithreading accelerates the process, and the wall time is roughly inversely proportional to the thread number.